# core

> Main codebase for fetching and saving RSS feeds

In [ ]:
# | default_exp core

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

import json
import logging
from pathlib import Path
from typing import *

import fire
import reader
import rich
from bs4 import BeautifulSoup
from fastcore.basics import patch_to
from rich import print
from rich.console import Console
from rich.panel import Panel
from rich.text import Text

console = Console()

logger = logging.getLogger(__name__)


def strip_html(s: str):
    s = BeautifulSoup(s, "html.parser")
    return s.text

In [ ]:
# | export

class Feed:
    """RSS feed class"""

    def __init__(self, url: str, name: str = None, tags: List[str] = []):
        self.url = url
        self.name = name
        self.tags = tags

    def add_tag(self, tag: str):
        if tag not in self.tags:
            self.tags.append(tag)

    def remove_tag(self, tag: str):
        if tag in self.tags:
            self.tags.remove(tag)

In [ ]:
# | export


class PyNewsReader:
    def __init__(self, feeds=List[Feed]):
        self._dbfolder = Path().home() / ".cache/pynewsreader"
        if not self._dbfolder.exists():
            self._dbfolder.mkdir(parents=True)

        # If custom feed names exist, read them in
        feed_names_path = self._dbfolder / "feed_names.json"
        if feed_names_path.exists():
            with open(self._dbfolder / "feed_names.json", "rt") as myfile:
                self._feed_names = json.load(myfile)
        else:
            self._feed_names = {}

        # If title blacklist exists, read it in
        title_blacklist_path = self._dbfolder / "title_blacklist.json"
        if title_blacklist_path.exists():
            with open(self._dbfolder / "title_blacklist.json", "rt") as myfile:
                self._title_blacklist = json.load(myfile)
        else:
            self._title_blacklist = []

        self._reader = reader.make_reader(
            self._dbfolder / "db.sqlite",
            plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"],
        )

        # If title whitelist exists, read it in
        title_whitelist_path = self._dbfolder / "title_whitelist.json"
        if title_whitelist_path.exists():
            with open(self._dbfolder / "title_whitelist.json", "rt") as myfile:
                self._title_whitelist = json.load(myfile)
        else:
            self._title_whitelist = []

        self._reader = reader.make_reader(
            self._dbfolder / "db.sqlite",
            plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"],
        )

        self._reader.enable_search()

    def _print_entries(
        self, entries: List[reader.Entry], mark_as_read: bool = True, limit: int = 10
    ):
        """Pretty print entries - supports reader.Reader.get_entries arguments"""
        displayed_links = set()
        for e in entries:
            if e.link in displayed_links:
                # Don't display duplicates
                self._reader.mark_entry_as_read(e)
            else:
                displayed_links.add(e.link)
                if e.published:
                    published_date = "Date: " + e.published.isoformat()[:10]
                else:
                    published_date = "Date: Unknown"
                if mark_as_read:
                    self._reader.mark_entry_as_read(e)

                feed_title = f"[bold]{self._get_feed_title(e.original_feed_url)}[/bold]"

                if e.important:
                    panel_body = ":exclamation_mark:"
                else:
                    panel_body = ""
                panel_body += f"Title: [bold]{e.title}[/bold]" + "\n"
                panel_body += str(published_date) + "\n\n"
                panel_body += strip_html(e.summary).strip() + "\n"

                console.print(
                    Panel(
                        panel_body,
                        title=feed_title,
                        subtitle=f"[link={e.link}]{e.link}[/link]",
                    )
                )
                console.print()
            if len(displayed_links) == limit:
                return

    def _get_feed_title(self, url: str):
        """Get display title for pynewsreader feed"""
        if url in self._feed_names and self._feed_names[url] is not None:
            return self._feed_names[url]
        elif self._reader.get_feed(url).title:
            return self._reader.get_feed(url).title
        else:
            return self._reader.get_feed(url).url

    def _get_entries(
        self, important: bool = None, read: Union[None, bool] = None, limit: int = 10
    ) -> List[reader.Entry]:
        """Get entries in reader.Entry format"""
        return self._reader.get_entries(read=read, limit=limit, important=important)

    def _get_tags(self, entry: reader.Entry):
        """Get tags for a given entry"""
        return [i[0] for i in list(self._reader.get_tags(entry))]

    def _mark_matching_entries_as_read(self, match_strings: List):
        for i in self._reader.get_entries(read=False):
            for filter_string in match_strings:
                if filter_string in i.title:
                    print(f"Marking entry as read: {i.title}")
                    self._reader.mark_entry_as_read(i)
                if filter_string in i.link:
                    print(f"Marking entry as read: {i.title}")
                    self._reader.mark_entry_as_read(i)

    def _mark_matching_entries_as_important(self, match_strings: List):
        for i in self._reader.get_entries(read=False):
            for filter_string in match_strings:
                if filter_string in i.title:
                    print(f"Marking entry as important: {i.title}")
                    self._reader.mark_entry_as_important(i)

    def _mark_important(self, entry: reader.Entry = None):
        """Mark entry as important"""
        if entry is not None:
            reader.Reader.mark_entry_as_important(entry)

    def _mark_unimportant(self, entry: reader.Entry = None):
        """Mark entry as important"""
        if entry is not None:
            reader.Reader.mark_entry_as_unimportant(entry)

    def _add_tag(self, entry: reader.Entry, tag_key: str, tag_value: Dict = None):
        """Add tag to entry"""
        reader.Reader.set_tag(entry, tag_key, tag_value)

    def _remove_tag(self, entry: reader.Entry, tag_key: str):
        """Remove tag from entry"""
        self._reader.delete_tag(entry, tag_key)

In [ ]:
# | export

@patch_to(PyNewsReader)
def blacklist_add(self, blacklist_string: str):
    """Add entry to blacklist"""
    if blacklist_string not in self._title_blacklist:
        self._title_blacklist.append(blacklist_string)
        with open(self._dbfolder / "title_blacklist.json", "wt") as myfile:
            json.dump(self._title_blacklist, myfile)


@patch_to(PyNewsReader)
def blacklist_remove(self, blacklist_string: str):
    """Remove entry from blacklist"""
    if blacklist_string in self._title_blacklist:
        self._title_blacklist.remove(blacklist_string)
        with open(self._dbfolder / "title_blacklist.json", "wt") as myfile:
            json.dump(self._title_blacklist, myfile)

@patch_to(PyNewsReader)
def blacklist_show(self):
    """Show blacklist"""
    print(self._title_blacklist)
  

In [ ]:
#| export

@patch_to(PyNewsReader)
def whitelist_add(self, whitelist_string: str):
    """Add entry to whitelist"""
    if whitelist_string not in self._title_whitelist:
        self._title_whitelist.append(whitelist_string)
        with open(self._dbfolder / "title_whitelist.json", "wt") as myfile:
            json.dump(self._title_whitelist, myfile)
        for entry in self._get_entries(limit=None):
            if whitelist_string in entry.title:
                self._reader.mark_entry_as_important(entry)


@patch_to(PyNewsReader)
def whitelist_remove(self, whitelist_string: str):
    """Remove whitelist entry"""
    if whitelist_string in self._title_whitelist:
        self._title_whitelist.remove(whitelist_string)
        with open(self._dbfolder / "title_whitelist.json", "wt") as myfile:
            json.dump(self._title_whitelist, myfile)
        for entry in self._get_entries(limit=None):
            if whitelist_string in entry.title:
                self._reader.mark_entry_as_unimportant(entry)
                
@patch_to(PyNewsReader)
def whitelist_show(self):
    """Show whitelist"""
    print(self._title_whitelist)
  

In [ ]:
# | export

@patch_to(PyNewsReader)
def update(self):
    """Update feeds and search"""
    self._reader.update_feeds()
    self._reader.update_search()
    if len(self._title_blacklist) > 0:
        self._mark_matching_entries_as_read(self._title_blacklist)
    if len(self._title_whitelist) > 0:
        self._mark_matching_entries_as_important(self._title_whitelist)

In [ ]:
# | export

@patch_to(PyNewsReader)
def add_feed(self, feed: Union[Feed, str]):
    """Add feed to pynewsreader

    Args:
        feed (Feed): pynewsreader Feed to add
    """
    if isinstance(feed, Feed):
        self._feed_names[feed.url] = feed.name
        self._reader.add_feed(feed.url, exist_ok=True)
    elif isinstance(feed, str):
        self._reader.add_feed(feed, exist_ok=True)
    else:
        raise Exception("Must be str or Feed type to add")

    # Save names to file
    with open(self._dbfolder / "feed_names.json", "wt") as myfile:
        json.dump(self._feed_names, myfile)


@patch_to(PyNewsReader)
def remove_feed(self, feed: Union[Feed, str]):
    """Remove feed from pynewsreader instance

    Args:
        feed (Union[Feed, str]): Feed to remove
    """

    if isinstance(feed, Feed):
        self._reader.delete_feed(feed.url)
    elif isinstance(feed, str):
        self._reader.delete_feed(feed)
    else:
        raise Exception(TypeError)


@patch_to(PyNewsReader)
def feeds(self):
    """List pynewsreader feeds

    Returns:
        List[str]: List of names of current pynewsreader feeds
    """
    return [self._get_feed_title(i.url) for i in self._reader.get_feeds()]

In [ ]:
# | export

@patch_to(PyNewsReader)
def show(
    self,
    limit: int = 6,
    read: bool = False,
    important: bool = None,
    mark_as_read: bool = True,
):
    """Pretty print entries

    Args:
        limit (int, optional): Number of entries to show. Defaults to 5.
        read (bool, optional): Show read entries (True), unread entries (False), or all entries (None). Defaults to None.
        mark_as_read (bool, optional): Mark displayed entries as read. Defaults to False.
    """
    self._print_entries(
        self._get_entries(read=read, important=important, limit=limit * 2),
        limit=limit,
        mark_as_read=mark_as_read,
    )

In [ ]:
# | export

@patch_to(PyNewsReader)
def search(self, query: str, mark_as_read: bool = True, limit: int = 10):
    """Search entries and pretty print results

    Args:
        query (str): Search query
        mark_as_read (bool, optional): Mark results as read? Defaults to True.
    """
    self._print_entries(
        [self._search_to_entry(i) for i in self._reader.search_entries(query)],
        mark_as_read=mark_as_read,
        limit=limit,
    )


@patch_to(PyNewsReader)
def _search_to_entry(self, search_result):
    """For a given `reader` search result return the corresponding `reader.Entry`"""
    for i in self._reader.get_entries():
        if i.id == search_result.id and i.feed_url == search_result.feed_url:
            return i

In [ ]:
r = PyNewsReader()

## Add Feeds

In [ ]:
# With names
for i in [
    Feed(url="https://ricochet.media/en/feed", name="Richochet Media"),
    Feed(url="https://thetyee.ca/rss2.xml", name="The Tyee"),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.topstories.rss",
        name="Toronto Star | Top Stories",
    ),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.news.investigations.rss",
        name="Toronto Star | Investigations",
    ),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.opinion.editorials.rss",
        name="Toronto Star | Editorials",
    ),
    Feed(url="https://www.macleans.ca/feed/", name="Macleans"),
]:
    r.add_feed(i)

# Use the default name from the feed
for i in [
    "https://rss.cbc.ca/lineup/topstories.xml",
    "https://rss.cbc.ca/lineup/world.xml",
    "https://rss.cbc.ca/lineup/canada.xml",
    "https://rss.cbc.ca/lineup/business.xml",
    "https://rss.cbc.ca/lineup/technology.xml",
    "https://www.cbc.ca/cmlink/rss-canada-ottawa",
    "https://thenarwhal.ca/feed/",
    "https://www.theguardian.com/science/rss",
    "https://www.theguardian.com/world/canada/rss",
]:
    r.add_feed(Feed(i))

## Update Feeds

In [ ]:
r.update()

Marking entry as read: Brooke Henderson takes three-shot lead into final round at Tournament of Champions

Marking entry as read: Elon Musk defends 2018 Tesla takeover tweet in class action trial testimony

Marking entry as read: Elon Musk defends 2018 Tesla takeover tweet in class action trial testimony

Marking entry as read: Alphabet — Google's corporate parent — to cut 12,000 jobs

Marking entry as read: Alphabet — Google's corporate parent — to cut 12,000 jobs

Marking entry as read: Tesla investors say Elon Musk's tweets cost them billions. His trial starts today

Marking entry as read: CF Montréal fire coach one day after hire due to assassination tweet

Marking entry as important: With inflation easing will interest rates hikes pause?

Marking entry as important: Majority of Canadian consumers and businesses expect a recession this year, Bank of 
Canada surveys find

Marking entry as important: Bank of Canada may lose billions in coming years, think tank warns

## List Feeds

In [ ]:
r.feeds()

['Canada | The Guardian',
 'CBC | Business News',
 'CBC | Canada News',
 'CBC | Ottawa News',
 'CBC | Technology News',
 'CBC | Top Stories News',
 'CBC | World News',
 'Macleans',
 'Richochet Media',
 'Science | The Guardian',
 'The Narwhal',
 'The Tyee',
 'Toronto Star | Investigations',
 'Toronto Star | Editorials',
 'Toronto Star | Top Stories']

## Display Entries

`read` = None shows all entries

`read` = False shows only unread entries

`read` = True shows only read entries

In [ ]:
r.show(limit=5, read=None, mark_as_read=False)

╭──────────────────────────────────────────── CBC | Top Stories News ─────────────────────────────────────────────╮
│ Title: Actor Jeremy Renner reveals he broke more than 30 bones in snowplow accident                             │
│ Date: 2023-01-21                                                                                                │
│                                                                                                                 │
│ Actor Jeremy Renner says he broke more than 30 bones in a snowplow accident on New Year's Day.                  │
│                                                                                                                 │
╰───── ]8;id=192538;https://www.cbc.ca/news/entertainment/jeremy-renner-snowplow-accident-broken-bones-1.6722214?cmp=rss\https://www.cbc.ca/news/entertainment/jeremy-renner-snowplow-accident-broken-bones-1.6722214?cmp=rss]8;;\ ──────╯

╭──────────────────────────────────────────── CBC | Top Stories News ─────────────────────────────────────────────╮
│ Title: François Legault criticizes Justin Trudeau over notwithstanding clause comments                          │
│ Date: 2023-01-21                                                                                                │
│                                                                                                                 │
│ Quebec Premier François Legault is criticizing Prime Minister Justin Trudeau for "attacking Quebec's democracy  │
│ and people" by proposing to limit the use of the notwithstanding clause.                                        │
│                                                                                                                 │
╰─ ]8;id=943391;https://www.cbc.ca/news/canada/montreal/fran%C3%A7ois-legault-trudeau-democracy-under-attack-1.6722077?cmp=rss\https://www.cbc.ca/news/canada/montreal/fran%C3%A7ois-legault-trudeau-democracy-under-attack-1.6722077?cmp=rss]8;;\─╯

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: Welcome to the Year of the Rabbit (in Culture)                                                           │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ Vancouver’s Lunar New Year parade returns to Chinatown after years of cancellations. We’re glad to be back.     │
│                                                                                                                 │
╰────────────────────────── ]8;id=672462;https://thetyee.ca/Culture/2023/01/20/Welcome-Year-Rabbit/\https://thetyee.ca/Culture/2023/01/20/Welcome-Year-Rabbit/]8;;\ ───────────────────────────╯

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: Donors Step Up After Vancouver Council Axes Funding to VANDU Art Program (in News)                       │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ Councillors said they had problems with how the drug user group handled a previous city contract.               │
│                                                                                                                 │
╰────── ]8;id=747710;https://thetyee.ca/News/2023/01/20/Donors-Step-Up-Vancouver-Council-Axes-Funding-VANDU-Art-Program/\https://thetyee.ca/News/2023/01/20/Donors-Step-Up-Vancouver-Council-Axes-Funding-VANDU-Art-Program/]8;;\ ──────╯

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: Escape from Crimea to Canada (in Culture)                                                                │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ A Ukrainian refugee shares his story of fleeing fear to navigate a fresh start in BC.                           │
│                                                                                                                 │
╰────────────────────────── ]8;id=353000;https://thetyee.ca/Culture/2023/01/20/Escape-Crimea-Canada/\https://thetyee.ca/Culture/2023/01/20/Escape-Crimea-Canada/]8;;\ ──────────────────────────╯

## Search Entries

In [ ]:
r.search("rate", mark_as_read=False)

╭─────────────────────────────────────────────── CBC | World News ────────────────────────────────────────────────╮
│ Title: U.S. inflation rate was 6.5% in December                                                                 │
│ Date: 2023-01-12                                                                                                │
│                                                                                                                 │
│ The inflation rate in the U.S. decelerated to 6.5 per cent in December, mostly due to much cheaper gasoline.    │
│                                                                                                                 │
╰─────────────────── ]8;id=672405;https://www.cbc.ca/news/business/us-inflation-december-1.6711469?cmp=rss\https://www.cbc.ca/news/business/us-inflation-december-1.6711469?cmp=rss]8;;\ ────────────────────╯

╭─────────────────────────────────────────── Toronto Star | Editorials ───────────────────────────────────────────╮
│ ❗Title: With inflation easing will interest rates hikes pause?                                                 │
│ Date: 2023-01-20                                                                                                │
│                                                                                                                 │
│ With some welcome downward movement on inflation, let’s hope then that we’re seeing an end to the Bank of       │
│ Canada’s string of rate hikes.                                                                                  │
│                                                                                                                 │
╰─ ]8;id=471297;https://www.thestar.com/opinion/editorials/2023/01/20/with-inflation-easing-will-interest-rates-hikes-pause.html\https://www.thestar.com/opinion/editorials/2023/01/20/with-inflation-easing-will-interest-rates-hikes-pause.ht]8;;\─╯

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: Inflation rate slows to 6.3% — but groceries are still going up at nearly twice that pace                │
│ Date: 2023-01-17                                                                                                │
│                                                                                                                 │
│ Canada's annual inflation rate cooled to 6.3 per cent in December, its lowest level since February.             │
│                                                                                                                 │
╰───────────────── ]8;id=572725;https://www.cbc.ca/news/business/canada-inflation-december-1.6716315?cmp=rss\https://www.cbc.ca/news/business/canada-inflation-december-1.6716315?cmp=rss]8;;\ ──────────────────╯

╭────────────────────────────────────────── Toronto Star | Top Stories ───────────────────────────────────────────╮
│ Title: Vacancy rate for downtown Toronto offices climbs to 13.6 per cent                                        │
│ Date: 2023-01-12                                                                                                │
│                                                                                                                 │
│ This week, online retail giant Shopify put its massive new office space at The Well onto the sublet market.     │
│                                                                                                                 │
╰─ ]8;id=378960;https://www.thestar.com/business/2023/01/12/vacancy-rate-for-downtown-toronto-offices-climbs-to-136-per-cent.html\https://www.thestar.com/business/2023/01/12/vacancy-rate-for-downtown-toronto-offices-climbs-to-136-per-cent.h]8;;\─╯

╭────────────────────────────────────────── Toronto Star | Top Stories ───────────────────────────────────────────╮
│ Title: No-shows, dropouts and asylum requests — these 10 schools have Canada’s highest rates of                 │
│ ‘non-compliance’ among international students                                                                   │
│ Date: 2023-01-12                                                                                                │
│                                                                                                                 │
│ Data suggests some issues around compliance and enforcement, but institutions say they can't control students   │
│ after they are accepted into classes.                                                                           │
│                                                                                                                 │
╰─ ]8;id=692386;https://www.thestar.com/news/canada/2023/01/12/no-shows-dropouts-and-asylum-requests-these-10-schools-have-canadas-highest-rates-of-non-compliance-among-international-students.html\https://www.thestar.com/news/canada/2023/01/12/no-shows-dropouts-and-asylum-requests-these-10-schools-have-can]8;;\─╯

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: Canada's economy added 104,000 jobs in December, blowing well past expectations                          │
│ Date: 2023-01-06                                                                                                │
│                                                                                                                 │
│ Canada's economy added 104,000 jobs in December as the unemployment rate fell slightly to 5.0 per cent,         │
│ Statistics Canada said Friday. But the gain wasn't reflected in working hours, and a hot labour market could    │
│ mean another rate hike from the central bank in January.                                                        │
│                                                                                                                 │
╰──────────────────── ]8;id=635527;https://www.cbc.ca/news/business/canada-jobs-december-1.6705569?cmp=rss\https://www.cbc.ca/news/business/canada-jobs-december-1.6705569?cmp=rss]8;;\ ────────────────────╯

╭───────────────────────────────────────── Toronto Star | Investigations ─────────────────────────────────────────╮
│ Title: Toronto parents less likely to see reduced daycare fees than almost anywhere else in Ontario             │
│ Date: 2022-12-22                                                                                                │
│                                                                                                                 │
│ For-profit daycares opted out of $10/day child care program at much higher rate than non-profits, a Star        │
│ analysis found.                                                                                                 │
│                                                                                                                 │
╰─ ]8;id=457479;https://www.thestar.com/news/investigations/2022/12/22/toronto-parents-less-likely-to-see-reduced-daycare-fees-than-almost-anywhere-else-in-ontario.html\https://www.thestar.com/news/investigations/2022/12/22/toronto-parents-less-likely-to-see-reduced-daycare-fees]8;;\─╯

╭────────────────────────────────────────── Toronto Star | Top Stories ───────────────────────────────────────────╮
│ Title: New chair of telecom regulator takes aim at Canada’s high internet prices                                │
│ Date: 2023-01-21                                                                                                │
│                                                                                                                 │
│ Vicky Eatrides, a lawyer and bureaucrat who held senior roles with the Competition Bureau, said she plans new   │
│ rules for the internet market.                                                                                  │
│                                                                                                                 │
╰─ ]8;id=653257;https://www.thestar.com/business/2023/01/21/what-has-the-crtc-done-for-me-new-chair-of-telecom-regulator-takes-aim-at-canadas-high-internet-prices.html\https://www.thestar.com/business/2023/01/21/what-has-the-crtc-done-for-me-new-chair-of-telecom-regulator-takes]8;;\─╯

╭──────────────────────────────────────────────── Richochet Media ────────────────────────────────────────────────╮
│ Title: ‘Learn to live with the virus,’ universities are telling teachers                                        │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ Allen had just gotten over a mild case of COVID-19 when the symptoms hit. “Everything started to implode. I     │
│ couldn’t stand up. Couldn’t walk. Being out in the heat, my heart rate skyrocketed to 140, 150 beats per        │
│ minute,” Allen said. “I’m an enduranc...                                                                        │
│                                                                                                                 │
╰──────── ]8;id=469086;https://ricochet.media/en/3912/the-message-to-university-teachers-learn-to-live-with-the-virus\https://ricochet.media/en/3912/the-message-to-university-teachers-learn-to-live-with-the-virus]8;;\ ─────────╯

╭─────────────────────────────────────────────────── Macleans ────────────────────────────────────────────────────╮
│ Title: My Prediction: Rent-to-own will transform how Canadians buy and sell homes                               │
│ Date: 2022-12-15                                                                                                │
│                                                                                                                 │
│ Archaic rules keep homeownership out of reach for millions. That’s about to change.                             │
│ The post My Prediction: Rent-to-own will transform how Canadians buy and sell homes appeared first on           │
│ Macleans.ca.                                                                                                    │
│                                                                                                                 │
╰─ ]8;id=190655;https://www.macleans.ca/year-ahead/my-prediction-rent-to-own-will-transform-how-canadians-buy-and-sell-homes/\https://www.macleans.ca/year-ahead/my-prediction-rent-to-own-will-transform-how-canadians-buy-and-sell-homes/]8;;\ ─╯

## Tag Entry

In [ ]:
test = list(r._reader.get_entries())

In [ ]:
r._reader.set_tag(test[0], "foobar")

In [ ]:
list(r._reader.get_tags(test[0]))

[('foobar', None)]

In [ ]:
# We added a method to just return the tag key:
r._get_tags(test[0])

['foobar']

## Remove tag from entry

In [ ]:
r._reader.delete_tag(test[0], "foobar")

In [ ]:
list(r._get_tags(test[0]))

[]

## Mark as Important/Unimportant

In [ ]:
r._reader.mark_entry_as_important(test[0])
r._reader.mark_entry_as_unimportant(test[0])

## Mark as Read / Unread

In [ ]:
r._reader.mark_entry_as_read(test[0])
r._reader.mark_entry_as_unread(test[0])

## Automatically mark entries as read

When updating feeds, entries with titles matching these strings will automatically be marked as read

In [ ]:
for i in [
    "Musk",
    "Apple",
    "Bezos",
    "Google",
    "Samsung",
    "iPhone",
    "iPad",
    "guardian.com/sport/",
    "guardian.com/football/",
    "thestar.com/sports",
]:
    r.add_to_blacklist(i)

In [ ]:
r.blacklist_show()

[
    'Musk',
    'Apple',
    'Bezos',
    'Google',
    'Samsung',
    'iPhone',
    'iPad',
    'guardian.com/sport/',
    'guardian.com/football/',
    'thestar.com/sports'
]

In [ ]:
# r.update()

## Automatically mark entries as important

In [ ]:
for i in ["interest rate", "Bank of Canada", "housing market"]:
    r.whitelist_add(i)

In [ ]:
r.whitelist_show()

['interest rate', 'Bank of Canada', 'housing market']

In [ ]:
# r.update()

## Low level access to data

In [ ]:
entries = r._get_entries(limit=1)
[i for i in entries][0]

Entry(id='1.6722214', updated=None, title='Actor Jeremy Renner reveals he broke more than 30 bones in snowplow accident', link='https://www.cbc.ca/news/entertainment/jeremy-renner-snowplow-accident-broken-bones-1.6722214?cmp=rss', author='The Associated Press', published=datetime.datetime(2023, 1, 21, 23, 14, 3, tzinfo=datetime.timezone.utc), summary='<img alt="File photo of Jeremy Renner, taken in November 2021" height="259" src="https://i.cbc.ca/1.6722217.1674341826!/fileImage/httpImage/image.jpg_gen/derivatives/16x9_460/file-photo-of-jeremy-renner-taken-in-november-2021.jpg" title="Jeremy Renner attends the &quot;Hawkeye&quot; Special Screening at AMC Lincoln Square Theater on November 22, 2021 in New York City." width="460" />                <p>Actor Jeremy Renner says he broke more than 30 bones in a snowplow accident on New Year\'s Day.</p>', content=(), enclosures=(), read=False, read_modified=datetime.datetime(2023, 1, 21, 23, 33, 38, 394852, tzinfo=datetime.timezone.utc), impo

## CLI

In [ ]:
# | export


def main():
    fire.Fire(PyNewsReader)

In [ ]:
# | export


if __name__ == "__main__":
    main()

NAME
    ipykernel_launcher.py -f /home/mark/.local/share/jupyter/runtime/kernel-09dfadad-262a-4a63-a09e-bcd46569e84b.json

SYNOPSIS
    ipykernel_launcher.py -f /home/mark/.local/share/jupyter/runtime/kernel-09dfadad-262a-4a63-a09e-bcd46569e84b.json COMMAND

COMMANDS
    COMMAND is one of the following:

     add_feed
       Add feed to pynewsreader

     add_to_blacklist
       Add entry to blacklist

     add_to_whitelist
       Add entry to whitelist

     blacklist_add
       Add entry to blacklist

     blacklist_remove
       Remove entry from blacklist

     blacklist_show
       Show blacklist

     feeds
       List pynewsreader feeds

     remove_feed
       Remove feed from pynewsreader instance

     remove_from_blacklist
       Remove entry from blacklist

     remove_from_whitelist
       Remove whitelist entry

     search
       Search entries and pretty print results

     show
       Pretty print entries

     update
       Update feeds and search

     whitelist_add

In [ ]:
# | hide

# To Do:
# * User interface

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()